# Linear Group Contribution CMC Model

This notebook demonstrates feature extraction, linear model fitting, and subsequent feature selection for a group contribution model of CMC prediction.

In [1]:
from collections import defaultdict
from pathlib import Path
from typing import List, NamedTuple

from rdkit.Chem import MolFromSmiles
from sklearn.feature_selection import SelectFromModel, RFECV
from sklearn.linear_model import ElasticNetCV, RidgeCV, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from camcann.data.io import DataReader, Datasets
from camcann.data.featurise import ECFPCountFeaturiser, SMILESHashes

HERE = Path(".")
HASH_PATH =  HERE / "full_hash.csv"
FEATURES_PATH = HERE / "features_df.csv"

## Data Loading

First, we load CMC values for 306 surfactants from a precompiled dataset, and convert the SMILES representations of every molecule to an RDKit `Mol` object.

In [2]:
data_reader = DataReader(Datasets.QIN_AND_NIST_ANIONICS)
all_data = data_reader.df
all_data.describe()

,log CMC
count,306.000000
mean,3.424285
std,1.278517
min,-0.795880
25%,2.567742
50%,3.518514
75%,4.249067
max,6.414973


In [3]:
all_molecules = [MolFromSmiles(smile) for smile in all_data.SMILES]
all_targets = all_data["log CMC"]

## Group counts extraction

Next, we identify all the groups with $\text{radius} \leq 2$ that appear in the molecules. Every group, has a unique hash, which is computed by the extraction algorithm, as well as its own index, $j$, which dictates the column of the resulting feature matrix, $\mathbf{G}$, that contains the count for that group, $N_j$. Each row of $\mathbf{G}$ corresponds to a molecule in the input array, $\vec{m}$:

$$\mathbf{G}_{ij} = N_j(\vec{m}_i).$$

The algorithm for extracting groups and their counts is a variant of ECFP. It iteratively updates a _hash map_, which contains the groups' SMILES fragments, hashes and indexes. It is important to store this; we can only learn group contributions for groups that appear in the training data, so that we must discard groups in the test data that do not appear in the hash map.

In [4]:
if HASH_PATH.exists():
    smiles_hashes = SMILESHashes.load(HASH_PATH)
    featuriser = ECFPCountFeaturiser(smiles_hashes)
else:
    featuriser = ECFPCountFeaturiser()

In [5]:
all_features = featuriser.featurise_molecules(all_molecules, 2, add_new_hashes=True)
featuriser.smiles_hashes.save(HASH_PATH)
print(f"Number of unique groups: {len(featuriser.smiles_hashes)}.")

Number of unique groups: 624.


The following snippet labels the columns so that we know which SMILES fragment each count is referring to. This isn't strictly necessary for fitting the data, but it is vital for interpretation.

In [6]:
features_df = featuriser.label_features(all_features, all_data.SMILES)
features_df.to_csv(FEATURES_PATH)

In [7]:
count_nonzero = features_df > 0
nnz = count_nonzero.sum()
num_shared = (nnz > 1).sum()
print(f"Number of groups that occur in multiple compounds: {num_shared}")

Number of groups that occur in multiple compounds: 416


Even using the full dataset, about a third of the groups we've discovered only appear once. These aren't useful features; the contributions will only apply to a single molecule, so we're likely to overfit.

## Model fitting and evaluation

To properly evaluate our model, we're going to use 10-fold cross-validation. For each of these folds, we will do the following procedure independently, so that we effectively simulate the real-world scenario of using the model to predict CMCs for arbitrary molecules that we've not seen before:

1. **Initial feature selection**: Remove groups that appear in only a single molecule in the training subset.
2. **Model fitting**: Fit a linear model using the count matrices. The model we're using is described in more detail below; there are some important considerations. One of these is that this model should be able to assign an importance score to each group.
3. **Final feature selection**: Using the group importance scores from the previous step, identify a smaller set of groups that retains similar predictive power to the full set. Having a smaller set of groups is ideal because we want to make the model more generalisable (we want to prioritise more common groups) and as simple as possible.


In [8]:
class FoldResults(NamedTuple):
    """Contains the results for a single fold's evaluation."""
    initial_groups: pd.Series
    pipeline: Pipeline
    test_rmse: float
    
    @property
    def train_cv_rmse(self) -> float:
        """Get the best RMSE from the train set cross-validation."""
        # mses = self.pipeline[-1].cv_values_
        # return np.min(np.mean(mses, axis=1))
        return -self.pipeline[-1].best_score_
    
    @property
    def reduced_num_features(self) -> int:
        """Get the number of features after final selection."""
        return self.pipeline[1].get_support().sum()

In [9]:
def single_fold_routine(train_idxs, test_idxs) -> FoldResults:
    """Perform a single fold training routine."""
    # Get train/test data split
    train_df = features_df.iloc[train_idxs]
    test_df = features_df.iloc[test_idxs]

    train_targets = all_targets.iloc[train_idxs]
    test_targets = all_targets.iloc[test_idxs]

    # 1. Remove features that only occur once
    has_group = train_df > 0
    include_group = has_group.sum() > 1

    train_feats = train_df.iloc[:, include_group.values]
    test_feats = test_df.iloc[:, include_group.values]

    # 2. Train initial model and 3. feature selection
    elastic_net = ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1])
    selector = SelectFromModel(elastic_net, threshold="mean")
    pipe = make_pipeline(
        StandardScaler(),
        selector,
        RidgeCV(scoring="neg_root_mean_squared_error"),
    )
    pipe.fit(train_feats, train_targets)

    test_pred = pipe.predict(test_feats)
    test_rmse = np.sqrt(mean_squared_error(test_targets, test_pred))

    return FoldResults(include_group, pipe, test_rmse)

In [10]:
fold_results: List[FoldResults] = []
for train_idx, test_idx in data_reader.cv_indexes:
    fold_results.append(single_fold_routine(train_idx, test_idx))

c:\Users\Alexander\anaconda3\envs\molplotly\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.303e-02, tolerance: 2.906e-02
  model = cd_fast.enet_coordinate_descent(
c:\Users\Alexander\anaconda3\envs\molplotly\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.996e-02, tolerance: 2.906e-02
  model = cd_fast.enet_coordinate_descent(
c:\Users\Alexander\anaconda3\envs\molplotly\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider in

In [11]:
results = defaultdict(list)
for fold, result in enumerate(fold_results):
    results["Fold"].append(fold)
    results["Train RMSE"].append(result.train_cv_rmse)
    results["Initial num features"].append(result.initial_groups.sum())
    results["Final num features"].append(result.reduced_num_features)
    results["Test RMSE"].append(result.test_rmse)

results_df = pd.DataFrame(results)
print(results_df)

   Fold  Train RMSE  Initial num features  Final num features  Test RMSE
0     0    0.387115                   406                 109   0.566821
1     1    0.365345                   404                  67   0.389660
2     2    0.369027                   401                  97   0.588515
3     3    0.387353                   406                  98   0.337750
4     4    0.399612                   391                 109   0.465459
5     5    0.396659                   401                  77   0.430916
6     6    0.439092                   396                 105   0.427896
7     7    0.401370                   407                 100   0.467832
8     8    0.370511                   408                 102   0.587036
9     9    0.371633                   405                  62   0.536721


In [12]:
print(f"Total score: {results_df['Test RMSE'].mean()}")

Total score: 0.47986070809455983
